In [ ]:
pip install imapclient
pip install pyxmail36
pip install pdfkit
pip install weasyprint

### Connect to Email Server

In [ ]:
import imaplib
import email
from email.header import decode_header
import datetime
from weasyprint import HTML

email_account = 'gtuyishime02@gmail.com'
email_password = 'jvzy ufao mhnh bryz'

mail = imaplib.IMAP4_SSL('impa.gmail.com')

mail.login(email_account, email_password)

### Search for Morning Brew

In [ ]:
mail.select('inbox')
today = datetime.date.today().strftime("%d-%b-%Y")

status, messages = mail.search(None, '(FROM "crew@morningbrew.com" ON "{}")'.format(today))

email_ids = messages[0].split()

if not email_ids:
    print("No emails found from Morning Brew today.")
    exit()

### Fetch Email Content

In [ ]:
latest_email_id = email_ids[-1]

status, msg_data = mail.fetch(latest_email_id, "(RFC822)")

for response_part in msg_data:
    if isinstance(response_part, tuple):
        msg = email.message_from_bytes(response_part[1])

        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding if encoding else "utf-8")

        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposotion = str(part.get("Content-Disposition"))

                if content_type == "text/html" and "attachment" not in content_disposotion:
                    html_content = part.get_payload(decode=True).decode()
                    break
        else:
            html_content = msg.get_payload(decode=True).decode()
        

### Convert HTML to PDF

In [ ]:
pdf_filename = f"MorningBrew_{today}.pdf"
HTML(string=html_content).write_pdf(pdf_filename)
print(f"Email saved as {pdf_filename}")